In [123]:
""" File to graph several signals using the Fourier Transform
    and Fourier Coefficients
    Author: Zander Blasingame
    Course: EE 321 """

import numpy as np
from scipy import signal
import plotly.plotly as py
import plotly.graph_objs as go

In [124]:
""" Function Definitions """

# Constants
N = 2000
SIGNAL_RANGE = 40
RESOLUTION = 100E-3  # 100ms

# define discrete unit impulse
d = lambda x: 1 if x == 0 else 0


# Construct signal from fourier coefficients
def construct_signal(X_k, period=2*np.pi, N=N):
    return lambda x: np.sum(X_k(k) * np.exp(2j*np.pi*k*x/period)
                            for k in range(-N, N, 1))


def mean_squared_error(signal_1, signal_2, N=N):
    return 1/N * np.sum(np.abs(signal_1 - signal_2)**2)

# Problems

## Problem 1a

In [103]:
%%latex
Given the following defintions for fourier coefficients, $X_k$

\begin{equation*}X_k = \delta(k) + \frac{1}{4}\delta(k-1) + \frac{1}{4}\delta(k+1) + \frac{1}{j2}\delta(k-2) - \frac{1}{j2}\delta(k+2)\end{equation*}

<IPython.core.display.Latex object>

In [120]:
""" Problem 1a """
X_k = lambda k: d(k) + 0.25*d(k-1) + 0.25*d(k+1)- 0.5j*d(k-2) - 0.5j*d(k+2)

# Construct and plot signal
x = construct_signal(X_k, 2)
t = np.arange(-SIGNAL_RANGE/10, SIGNAL_RANGE/10, RESOLUTION)

data = [go.Scatter(x=t, y=x(t).real)]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 1a')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='signal_1.png')

## Problem 1b

In [73]:
%%latex
Given the following definitions for the Fourier coefficients, $X_k$

\begin{equation*}X_k = \bigg\{\begin{matrix}
               &jk \; &|k| < 3\\
               &0 &\text{otherwise}
               \end{matrix}
\end{equation*}



<IPython.core.display.Latex object>

In [42]:
""" Problem 1b """
X_k = lambda k: 1j*k if abs(k) < 3 else 0

# Construct and plot signal
x = construct_signal(X_k)
t = np.arange(-SIGNAL_RANGE/2, SIGNAL_RANGE/2, RESOLUTION)

data = [go.Scatter(x=t, y=x(t).real)]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 1b')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='signal_2.png')

# Problem 2

In [115]:
%%latex

Let the reconstructed signal, $\hat{x}(t)$ given $N$ components be defined as follows
\begin{equation}
\hat{x}(t) = \sum_{k=-N}^N X_ke^{\frac{j2\pi k}{T}t}
\end{equation}
Moreover, let the mean squared error between the original signal, $x(t)$ and the reconstructed signal be given by
\begin{equation}
\mathbb{E}(N) = \frac{1}{T} \int_0^T |x(t) - \hat{x}(t)|^2 dt
\end{equation}
In a discrete form this is represented as
\begin{equation}
\mathbb{E}(N) = \frac{1}{N} \sum_{n=1}^N |x(n) - \hat{x}(n)|^2
\end{equation}
Let the complex Fourier coefficients be expressed as
\begin{equation*}X_k = \frac{1}{T} \int_{t_0}^{t_0 + \frac{T}{2}} x(t)e^{-j\frac{2\pi kt}{T}}dt\end{equation*}

<IPython.core.display.Latex object>

## Problem 2a

In [48]:
%%latex
Let $x_1(t)$ be defined as

\begin{equation*}x_1(t) = \sum_{n=-\infty}^{\infty} u(t-3n) - u(t - 3n - 2)\end{equation*}
where $u(n)$ is the Heaviside step function. This can be rewritten as
\begin{equation*}x_1(t) = \sum_{n=-\infty}^{\infty} \text{rect}\bigg(\frac{t}{2} - \frac{3n + 1}{2}\bigg)\end{equation*}
To find the complex Fourier coefficients look at two time ranges: [-1, 0] and [0, 2] such that
\begin{equation*}X_k = \frac{1}{3} \int_{-1}^{2} x_1(t)e^{-j\frac{2\pi kt}{3}}dt\end{equation*}
This can be simplified to
\begin{equation*}X_k = \frac{1}{3}\bigg[ \int_{-1}^{0} x_1(t)e^{-j\frac{2\pi kt}{3}}dt
        + \int_{0}^{2} x_1(t)e^{-j\frac{2\pi kt}{3}} dt\bigg]\end{equation*}
\begin{equation*}X_k = \frac{1}{3}\bigg[ \int_{-1}^{0} 0e^{-j\frac{2\pi kt}{3}}dt
        + \int_{0}^{2} 1e^{-j\frac{2\pi kt}{3}} dt\bigg]\end{equation*}
\begin{equation*}X_k = \frac{1}{3} \int_{0}^{2} e^{-j\frac{2\pi kt}{3}}dt\end{equation*}
\begin{equation*}X_k = \frac{j}{2\pi k}\bigg[e^{-j\frac{4\pi k}{3}} - 1 \bigg]\end{equation*}
$X_0$ is calculated by evaluating the following limit
\begin{align*}
X_0 &= \lim_{k \to 0} \frac{j}{2\pi k}\bigg[e^{-j\frac{4\pi k}{3}} - 1\bigg]
\end{align*}
By using L'Hospital's Rule
\begin{align*}
X_0 &= \lim_{k \to 0} \frac{j}{2\pi k}\bigg[\frac{-j4\pi k}{3}e^{-j\frac{4\pi k}{3}} - 0\bigg]\\
X_0 &= \lim_{k \to 0} \frac{2}{3}e^{-j\frac{4\pi k}{3}}\\
X_0 &= \frac{2}{3}
\end{align*}
Thus $X_k$ can be rewritten as
\begin{equation*}X_k = \frac{j}{2\pi k}\bigg[e^{-j\frac{4\pi k}{3}} - 1 \bigg] + \frac{2}{3}\delta(k)\end{equation*}


<IPython.core.display.Latex object>

In [125]:
""" Problem 2a """
# added term to avoid dividing by zero
X_k = lambda k: 1j/(2*np.pi*k + 1E-9) * (np.exp(-1j*4*np.pi*k/3) - 1) + 2/3*d(k)

# Construct and plot signal
x = construct_signal(X_k, 3)
t = np.arange(0.001, 6, 0.01)

data = [go.Scatter(x=t, y=x(t).real)]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 2a')

fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename='signal_3.png')


In [126]:
# ideal signal
x_ideal = lambda t: signal.square(2/3*np.pi*t, duty=2/3)/2 + 1/2
t = np.arange(0.001, 6, 0.01)

num_components = np.arange(1,500,10)
errors = [mean_squared_error(construct_signal(X_k, 3, N=n)(t), x_ideal(t), N=n)
          for n in num_components]

data = [go.Scatter(x=num_components, y=errors)]
layout = go.Layout(xaxis=dict(title='N', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=25),
                   yaxis=dict(title='Error',
                              type='log'),
                   title='Reconstruction Error 2a')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='rec_err_1.png')

In [131]:
%%latex
Using the figure above it is found that the mean squared error equals 0.01 at roughly 80 components and it equals 0.001 at roughly 325 components.

<IPython.core.display.Latex object>

## Problem 2b

In [107]:
%%latex
Let $x_2(t)$ be defined as

\begin{equation*}x_2(t) = \sum_{n=-\infty}^{\infty} \text{tri}(t-(2n+1))\end{equation*}
where $\text{tri}(t)$ is the triangular pulse. This can be expressed as
\begin{equation}
\text{tri}(t) = \text{rect}(t/2)(1-|t|)
\end{equation}
The Fourier coefficients can be expressed as
\begin{equation}
X_k = \frac{1}{2} \int_{-1}^{1} \text{tri}(t)\cdot e^{-j\pi kt}dt
\end{equation}
Further simplification yields the following
\begin{align*}
X_k &= \frac{1}{2}\bigg[\int_{-1}^{0} -t \cdot e^{-j\pi kt}dt
        + \int_{0}^{1} t \cdot e^{-j\pi kt}dt\bigg]
\end{align*}
which evaluates to
\begin{align*}
X_k &= \frac{e^{-j\pi k}\big(-j\pi k + e^{j\pi k} - 1\big)
             + e^{j\pi k}\big(j\pi k - 1\big) + 1}{-2\pi^2k^2}\\
X_k &= \frac{e^{-j\pi k}\big(j\pi k - e^{j\pi k} + 1\big)
             - e^{j\pi k}\big(j\pi k - 1\big) - 1}{2\pi^2k^2}\\
X_k &= \frac{e^{-j\pi k}\big(j\pi k + 1\big)
             - e^{j\pi k}\big(j\pi k - 1\big) - 2}{2\pi^2k^2}\\
X_k &= \frac{j\pi k\big(e^{j\pi k} - e^{-j\pi k}\big) + \big(e^{j\pi k} + e^{-j\pi k}\big) - 2}{2\pi^2k^2}\\
X_k &= \frac{j\pi k\big(2j\sin(\pi k)\big) + \big(2\cos(\pi k)\big) - 2}{2\pi^2k^2}\\
X_k &= \frac{2\cos(\pi k) -2\pi k\sin(\pi k)  - 2}{2\pi^2k^2}\\
\end{align*}
It can be shown that $\cos(\pi k) = (-1)^k\; \forall k \in \mathbb{Z}$ and $\sin(\pi k) = 0\; \forall k \in \mathbb{Z}$.
Therefore the Fourier coefficients become
\begin{equation}
X_k = \frac{-1 + (-1)^k}{\pi^2k^2}
\end{equation}

To find $X_0$ evaulate the integral definition for the Fourier coefficients
with the limit $k\to 0$ such that
\begin{align*}
X_0 &= \lim_{k\to 0} \frac{1}{2}\bigg[\int_{-1}^{0} -t \cdot e^{-j\pi kt}dt
        + \int_{0}^{1} t \cdot e^{-j\pi kt}dt\bigg]\\
X_0 &= \frac{1}{2}\bigg[\int_{-1}^{0} -t dt
        + \int_{0}^{1} t dt\bigg]\\
X_0 &= \frac{1}{2}\bigg[\frac{-t^2}{2}\bigg|_{-1}^0
        + \frac{t^2}{t}\bigg|_0^1\bigg]\\
X_0 &= \frac{1}{2}
\end{align*}
Therefore, $X_k$ can be written as
\begin{equation}
X_k = \frac{e^{-j\pi k}\big(j\pi k - e^{j\pi k} + 1\big)
             - e^{j\pi k}\big(j\pi k - 1\big) - 1}{2\pi^2k^2} + \frac{1}{2}\delta(k)
\end{equation}

<IPython.core.display.Latex object>

In [127]:
""" Problem 2b """    
X_k = lambda k: (-1 + (-1)**k)/(np.pi**2 * (k+1E-9)**2)

# Construct and plot signal
x = construct_signal(lambda k: X_k(k) + 0.5*d(k), 2)
t = np.arange(0.001, 2, 0.01)

data = [go.Scatter(x=t, y=[x(time).real for time in t])]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 2b')

fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename='signal_4.png')


In [134]:
# ideal signal
x_ideal = lambda t: 2 * np.abs(0.5*t - np.floor(0.5*t + 0.5))
t = np.arange(0.001, 2, 0.01)

num_components = np.arange(1,50,5)
errors = [mean_squared_error(construct_signal(lambda k: X_k(k) + 0.5*d(k), 2, N=n)(t), x_ideal(t), N=n)
          for n in num_components]

data = [go.Scatter(x=num_components, y=errors)]
layout = go.Layout(xaxis=dict(title='N', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Error',
                              type='log'),
                   title='Reconstruction Error 2b')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='rec_err_2.png')

In [135]:
%%latex
Using the figure above it was found that the mean squared error equaled 0.01 at roughly 5 components and it equaled 0.001 at roughly 7 components. 

<IPython.core.display.Latex object>